In [0]:
# Day 1 table
bronze_table = "workspace.ecommerce.ecommerce_delta"

# Data read
df_bronze = spark.table(bronze_table)

print(f"Total rows in Bronze: {df_bronze.count()}")

In [0]:
from pyspark.sql.functions import count, sum, when, col, round

# Null user_id remove
df_clean = df_bronze.filter(col("user_id").isNotNull())

# User-level features create
df_features = (
    df_clean
    .groupBy("user_id") 
    .agg(
        count("*").alias("total_interactions"),
        sum(when(col("event_type") == "view", 1).otherwise(0)).alias("total_views"),
        sum(when(col("event_type") == "cart", 1).otherwise(0)).alias("total_cart_adds"),
        sum(when(col("event_type") == "purchase", 1).otherwise(0)).alias("total_purchases"),
        round(sum(when(col("event_type") == "purchase", col("price")).otherwise(0)), 2).alias("total_spend")
    )
)

display(df_features.limit(5))

In [0]:
silver_table_name = "workspace.ecommerce.user_features_silver"

df_features.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(silver_table_name)

print("Silver table successfully created and saved")

In [0]:
df_silver = spark.table(silver_table_name)

total_rows = df_silver.count()
distinct_users = df_silver.select("user_id").distinct().count()

print(f"Total Rows: {total_rows}")
print(f"Distinct Users: {distinct_users}")

# Validation Check
if total_rows == distinct_users:
    print("No duplicate users found")
else:
    print("Duplicates exist")

In [0]:
from pyspark.sql.functions import col, sum as _sum, when

# Validity check

silver_table_name = "workspace.ecommerce.user_features_silver"
df_silver = spark.table(silver_table_name)

total_rows = df_silver.count()
distinct_users = df_silver.select("user_id").distinct().count()
duplicate_count = total_rows - distinct_users

if duplicate_count == 0:
    print("DUPLICATES Available")
else:
    print(f"DUPLICATES:({duplicate_count} duplicates found)")

# Null Value Check
null_counts = df_silver.select([
    _sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df_silver.columns
]).collect()[0].asDict()

has_nulls = False
for column, count in null_counts.items():
    if count > 0:
        print(f"NULLS: Failed -> Column '{column}' has {count} NULL values.")
        has_nulls = True
if not has_nulls:
    print("NULLS: Passed (No missing values)")

# Negative Value Check
negative_interactions = df_silver.filter(col("total_interactions") < 0).count()
negative_purchases = df_silver.filter(col("total_purchases") < 0).count()

if negative_interactions == 0 and negative_purchases == 0:
    print("No negative values in metrics")
else:
    if negative_interactions > 0:
        print(f"{negative_interactions} rows with negative interactions.")
    if negative_purchases > 0:
        print(f"{negative_purchases} rows with negative purchases.")